In [1]:
%matplotlib inline

In [5]:
import skimage.io
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, writers
import pandas as pd
import ipympl
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import math
import sys, os
sys.path.append('/sise/home/shakarch/muscle-formation-regeneration')
sys.path.append(os.path.abspath('..'))
from TimeSeriesAnalysis.utils.data_load_save import *

plt.rcParams["animation.html"] = "jshtml"
# plt.rcParams['animation.ffmpeg_path'] = "C:\\FFmpeg\\bin\\ffmpeg"
plt.rcParams['figure.dpi'] = 80  
plt.rcParams['animation.embed_limit'] = 2**128
plt.rcParams['figure.figsize']=(12,12)

<h2>Prepare Data<h2>

In [6]:
def load_vid(videoPath):
    video_tif_nuc = skimage.io.imread(videoPath)
    video_np = np.array(video_tif_nuc)
    
    return video_np


In [7]:
# load data
tracks = get_all_properties_df("actin_intensity", 1, 5, scores_vid_num=3)
video_np = load_vid("/storage/users/assafzar/Muscle_Differentiation_AvinoamLab/data/videos/test/S3_nuclei.tif") # 211212erki-p38-stiching_s3_EGFP_ORG

<h2>Creating new data frame with Sma and Discretization columns<h2>

In [8]:
def create_sma_column(df, col_to_calculate, w_size_sma):
    ID_tracks = [group for _, group in df.groupby('Spot track ID')]
    tracks_SMA = []
    
    for ID in ID_tracks:
        score_df = ID
        score_df[col_to_calculate + '_SMA'] = score_df[col_to_calculate].rolling(w_size_sma).mean()
        tracks_SMA.append(score_df)

    All_tracks_SMA = pd.concat(tracks_SMA)
    
    return All_tracks_SMA

def Create_Discretization_Column(dataFrame, columnToCalculate, pathToSave):
    
    def bins(num):
        floor_num = (math.floor(num * 10)) /10
        return floor_num
    
    dataFrame[columnToCalculate] = All_tracks_SMA[columnToCalculate].fillna(0)
    dataFrame['score_discretization'] = dataFrame['score_intensity']
    ID_tracks = [group for _, group in All_tracks_SMA.groupby('Spot track ID')]
    tracks_discretization = []
    
    for ID in ID_tracks:   
        score_df = ID
        score = ID.iloc[:,7]
        score_disc = [bins(item) for item in score]

        score_df['score_discretization'] = score_disc
        tracks_discretization.append(score_df)

    All_tracks_bins_SMA = pd.concat(tracks_discretization)
    df_to_pickle = All_tracks_bins_SMA.to_pickle(pathToSave)
    
    return df_to_pickle

def Create_Sma_And__Discretization_Column(dataFrame, columnToCalculate, windowSizeSma, pathToSave):
    df_with_sma = Create_Sma_Column(dataFrame, columnToCalculate, windowSizeSma)
    df_with_disc_sma = Create_Discretization_Column(df_with_sma, columnToCalculate, pathToSave)
    
    return df_with_disc_sma

def create_change_in_score_column(df, col_to_calculate): # todo group by Spot track ID & sory by Spot frame
    df[f"{col_to_calculate}_change"] = df[col_to_calculate].diff()
    return df


def convert_to_groupByFrame(df):
    tracks = [group for _, group in df.groupby('Spot frame')]
    
    return tracks



<h2>Visualization<h2>

In [9]:
def VisualizationAnimation(tracks_df, present_col_name, GifName, video, frame_start=0, frame_end=258, cmap="Oranges", norm_vals=(0, 1)):
    fig, ax = plt.subplots(figsize=(15,13))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    div = make_axes_locatable(ax)
    cax = div.append_axes('right', '3%', '5%')
    tracks_df = tracks_df[(tracks_df["Spot frame"]>=frame_start) & (tracks_df["Spot frame"]<=frame_end)]
    tracks_by_frame = convert_to_groupByFrame(tracks_df)

    def animate(i):
        frame = int(i["Spot frame"].min())
        x_val = (i["Spot position X"]) / 0.462
        y_val = (i["Spot position Y"]) / 0.462
        color = i[present_col_name]
        print(frame)
        
        cax.cla()
        ax.cla()
        ax.set_title("Time: {hour:.2f} (h)".format(hour = frame * (1/12)), fontsize=25, color="orange")

        frame_image = video_np[frame] 
        ax.imshow(frame_image, vmin=0, vmax=255, cmap='gist_gray')

        im = ax.scatter(x_val,y_val,s=50,c=color, cmap=cmap, edgecolor='black', linewidth=0.75, alpha=0.75, norm=mpl.colors.Normalize(norm_vals[0], norm_vals[1]))
        plt.colorbar(im, cax=cax, ticks=np.arange(0,1,0.1))
    
    anim = FuncAnimation(fig,animate,frames=tracks_by_frame,interval=100)

    writergif = animation.PillowWriter(fps=10)
    anim.save(GifName,writer=writergif)

    plt.close()

    

In [1]:
tracks = create_sma_column(tracks, "score_intensity", 5)
tracks = create_change_in_score_column(tracks, "score_intensity_SMA")
tracks = create_sma_column(tracks, "score_intensity_SMA_change", 5)

VisualizationAnimation(tracks, "score_intensity_SMA_change_SMA", 'Diff_visualization_sma_change_sma.gif', video_np, 140, 258, cmap="PiYG", norm_vals=(-0.05,0.05))

NameError: name 'create_sma_column' is not defined